In [6]:
import json
import time
from typing import Dict

import numpy as np
import pandas as pd
from power_grid_model import CalculationMethod, CalculationType, PowerGridModel, initialize_array
from power_grid_model.utils import json_deserialize, json_serialize
from power_grid_model.validation import assert_valid_batch_data, assert_valid_input_data

with open("data/input/input_network_data.json") as fp:
    input_data = json_deserialize(fp.read())
assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.power_flow)
model = PowerGridModel(input_data=input_data)

active_power_profile = pd.read_parquet("data/input/active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("data/input/reactive_power_profile.parquet")

# Create a PGM batch update dataset with the active and reactive load profiles.

load_profile = initialize_array("update", "sym_load", (active_power_profile.shape))
# active_power_profile.shape) gives a column too much
load_profile["id"] = active_power_profile.columns.to_numpy()
load_profile["p_specified"] = active_power_profile.to_numpy()
load_profile["q_specified"] = reactive_power_profile.to_numpy()
update_data = {"sym_load": load_profile}
display(load_profile)
assert_valid_batch_data(input_data=input_data, update_data=update_data, calculation_type=CalculationType.power_flow)

# Run Newton-Raphson power flow
output_data = model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.newton_raphson)

# Run linear power flow
output_data_linear = model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.linear)

array([[( 8, -128,   97627.00785465, -470888.77579075),
        ( 9, -128,  430378.73274484,  548467.37886843),
        (10, -128,  205526.75214329,  -87699.3355669 )],
       [( 8, -128,   89766.36599379,  136867.8977373 ),
        ( 9, -128, -152690.40132219, -962420.39912729),
        (10, -128,  291788.22613331,  235270.99415175)],
       [( 8, -128, -124825.57747461,  224191.44544484),
        ( 9, -128,  783546.00156416,  233867.99374951),
        (10, -128,  927325.52100206,  887496.15702925)],
       [( 8, -128, -233116.96234844,  363640.59820697),
        ( 9, -128,  583450.07616533, -280984.19885243),
        (10, -128,   57789.83950581, -125936.09240132)],
       [( 8, -128,  136089.12218786,  395262.39185453),
        ( 9, -128,  851193.27658532, -879549.05674146),
        (10, -128, -857927.88360423,  333533.43089134)],
       [( 8, -128, -825741.40059692,  341275.73923632),
        ( 9, -128, -959563.20511935, -579234.87785232),
        (10, -128,  665239.69109588, -74214